In [1]:
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5 import *
from deepface import DeepFace 
import sys
import os
import cv2
import pandas as pd

In [2]:
#threshold value is collected from slider, but no deepface yet
threshold = 50

#setting up data for results
data = pd.DataFrame()

data = pd.concat([_pic, _age, _gender, _race, _white, _black, _latino_hispanic, _asian, _indian, _middle_eastern], axis=1)
data.columns = ['Source', 'Age', "Gender", "Race", "White", "Black", "Latino/Hispanic", "Asian", "Indian", "MiddleEastern"]

class QComboBox(QtWidgets.QComboBox):
    def __init__(self, parent=None):
        super(QIComboBox, self).__init__(parent)
        
class Wizard(QtWidgets.QWizard):
    
    #redefining nextId
    def nextId(self):
        id = self.currentId()
        if id == 2:
            if self.page2.batch_cb.isChecked():
                return 5
            else:
                return 3
        if id == 1:
            return 2
        if id == 3:
            return 4
        if id == 5:
            return 6
        # ensures no next button - finishes on either of these based on check boxes
        if id == 6 or id == 4:
            return -1

    def __init__(self, parent=None):
        super(Wizard, self).__init__(parent)

        #add page 1,2
        self.page1 = Page1()
        self.setPage(1, self.page1)
        
        self.page2 = Page2()
        self.setPage(2, self.page2)
        
        self.setStartId(1)
        
        #set ids for all potential pages
        #id = 3
        self.page3single = Page3Single()
        self.page3singleid = self.setPage(3, self.page3single)
        
        #id = 4
        self.page4single = Page4Single()
        self.page4single.setFinalPage(True)
        self.page4singleid = self.setPage(4, self.page4single)
        
        #id = 5
        self.page3batch = Page3Batch()
        self.page3batchid = self.setPage(5, self.page3batch)
        
        #id = 6
        self.page4batch = Page4Batch()
        self.page4batch.setFinalPage(True)
        self.page4batchid = self.setPage(6, self.page4batch)
        
        self.setWindowTitle("DCiFR")
        self.setGeometry(0, 0, 800, 600)
    
# page 1 - select desired attributes for analyzing
class Page1(QtWidgets.QWizardPage):
    def ValueContrast(self, value):
        threshold = value
        
    def __init__(self, parent=None):
        super(Page1, self).__init__(parent)
        
        self.title_label = QLabel('Welcome to DCiFR!', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()

        self.title_label = QLabel('Attributes', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()

        #hover info 
        info = QLabel('Check the boxes that apply. Hover for more info!', self)
        info.move(50, 125)
        info.setFont(QFont('Arial', 10))
        info.adjustSize()
        myFont=QtGui.QFont()
        myFont.setItalic(True)
        info.setFont(myFont)
        info.adjustSize()
        
        #Check boxes
        age_cb = QCheckBox('Age', self)
        age_cb.move(50, 150)
        race_cb = QCheckBox('Race', self)
        race_cb.move(50, 200)
        age_cb.adjustSize()
        race_cb.adjustSize()
        gender_cb = QCheckBox('Gender', self)
        gender_cb.move(50, 250)
        emotion_cb = QCheckBox('Emotion', self)
        emotion_cb.move(50, 300)
        gender_cb.adjustSize()
        emotion_cb.adjustSize()
        
        #Adding slider for race attribute
        self.sld = QSlider(Qt.Horizontal, self)
        self.sld.setRange(0, 100)
        self.sld.move(500, 240)
        self.sld.setTickInterval(10)
        label = QLabel('Please select your threshold value \n(only applicable if you select the race attribute)', self)
        label.move(300, 150)
        minlabel = QLabel('0', self)
        minlabel.move(500, 260)
        maxlabel = QLabel('100', self)
        maxlabel.move(570, 260)
        q1label = QLabel('25', self)
        q1label.move(510, 260)
        medlabel = QLabel('50', self)
        medlabel.move(530, 260)
        q3label = QLabel('75', self)
        q3label.move(550, 260)
        minlabel.setFont(QFont('Arial', 5))
        maxlabel.setFont(QFont('Arial', 5))
        q1label.setFont(QFont('Arial', 5))
        medlabel.setFont(QFont('Arial', 5))
        q3label.setFont(QFont('Arial', 5))
        minlabel.adjustSize()
        maxlabel.adjustSize()
        q1label.adjustSize()
        medlabel.adjustSize()
        q3label.adjustSize()
        
        # tool tip
        self.sld.setToolTip('This is a slider for the threshold of the race attribute.')
            # need to connect slider value to changing threshold value
        self.sld.valueChanged[int].connect(self.ValueContrast)
        
        #Hovers
        age_cb.setToolTip('Check this box if you would like to analyze the age of the subject in your image(s)')
        race_cb.setToolTip('Check this box if you would like to analyze the race of the subject in your image(s)')
        gender_cb.setToolTip('Check this box if you would like to analyze the gender of the subject in your image(s)')
        emotion_cb.setToolTip('Check this box if you would like to analyze the emotion of the subject in your image(s)')
        
        age_cb_checked = age_cb.isChecked()
        gender_cb_checked = gender_cb.isChecked()
        emotion_cb_checked = emotion_cb.isChecked()
        race_cb_checked = race_cb.isChecked()
        
#page 2 - choose single or batch mode        
class Page2(QtWidgets.QWizardPage):
    def __init__(self, parent=None):
        super(Page2, self).__init__(parent)
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()
        
        self.title_label = QLabel('Single or Batch Mode', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()
        #hover info 
        info = QLabel('Check the box that applies. Hover for more info!', self)
        info.move(50, 125)
        info.setFont(QFont('Arial', 10))
        info.adjustSize()
        myFont=QtGui.QFont()
        myFont.setItalic(True)
        info.setFont(myFont)
        info.adjustSize()
        
        #Check boxes - hbox allows these to be exclusive
        hbox = QHBoxLayout()
    
        self.single_cb = QCheckBox('Single Image', self)
        self.single_cb.move(50, 150)
        self.batch_cb = QCheckBox('Batch Mode', self)
        self.batch_cb.move(50, 200)
        self.single_cb.adjustSize()
        self.batch_cb.adjustSize()

        group = QButtonGroup(self)
        group.addButton(self.single_cb)
        group.addButton(self.batch_cb)
        
        hbox.addWidget(self.single_cb)
        hbox.addWidget(self.batch_cb)
        
        #Hovers
        self.single_cb.setToolTip('Check this box if you would like to analyze demograhpics for a single image')
        self.batch_cb.setToolTip('Check this box if you would like to analyze demograhpics for more than one image')

# file upload
class Page3Single(QtWidgets.QWizardPage):
    #upload and analyze a single image
    
    # DeepFace integration
    # Checks for one face, and if there is ONLY one face, will perform analysis
    
    #TO-DO: add if/else conditions based on checkbox statuses
        # need way to reference values from page 1
    
    def get_image_file(self):
            dialog = QFileDialog()
            file_name = dialog.getOpenFileName(self, 'Open image')
            file = os.path.join(file_name[0])
            print(file)
            self.detect_face_show(file)
            
    def analyze(self, reuslts):
        # i think im doing this like its multiple images? i can work back tho
        age_ = []
        gender_ = []
        race_ = []
        emotion_ = []
        asian_ = []
        black_ = []
        indian_ = []
        latino_hispanic_ = []
        middle_eastern_ = []
        white_ = []
        
        for i in range(len(results)):
            age = results[i]['age']
            gender = results[i]['gender']
            race = results[i]['dominant_race']
            emotion = results[i]['emotion']
            asian = results[i]['race']['asian']
            black = results[i]['race']['black']
            indian = results[i]['race']['indian']
            latino_hispanic = results[i]['race']['latino hispanic']
            middle_eastern = results[i]['race']['middle eastern']
            white = results[i]['race']['white']
        
            age_.append(age)
            gender_.append(gender)
            race_.append(race)
            emotion_.append(emotion)
            black_.append(black)
            indian_.append(indian)
            latino_hispanic_.append(latino_hispanic)
            middle_eastern_.append(middle_eastern)
            white_.append(white)
        
        _pic = pd.DataFrame(pics)
        _age = pd.DataFrame(age_)
        _gender = pd.DataFrame(gender_)
        _emotion = pd.DataFrame(emotion_)
        _race = pd.DataFrame(race_)
        _white = pd.DataFrame(white_)
        _black = pd.DataFrame(black_)
        _latino_hispanic = pd.DataFrame(latino_hispanic_)
        _asian = pd.DataFrame(asian_)
        _indian = pd.DataFrame(indian_)
        _middle_eastern = pd.DataFrame(middle_eastern_)
        
        data = pd.concat([_pic, _age, _gender, _race, _white, _black, _latino_hispanic, _asian, _indian, _middle_eastern], axis=1)
            
    def detect_face_show(self, fpath):
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        img = cv2.imread(fpath)
        if img is None:
            return(0)
        else:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)
            face_num = len(faces)
            data['filepath'] = pd.Series(fpath)
            filename = fpath.split("\\")
            data['id'] = pd.Series(filename)
            data['faces'] = face_num
            if (face_num == 1):
                # running into problems - i think it's being read in incorrectly???
                result = DeepFace.analyze(faces, ['race'], enforce_detection=False)
                #analyze(results)
                print(results)
                print("done!")
            else:
                return(0)
            
    def __init__(self, parent=None):
        super(Page3Single, self).__init__(parent)
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()
        self.title_label = QLabel('Upload Your Image Below', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 10))
        self.title_label.adjustSize()
        
        self.button1 = QPushButton("Select An Image to Upload Here", self)  
        self.button1.clicked.connect(self.get_image_file) 
        self.button1.move(50, 150)
        
class Page3Batch(QtWidgets.QWizardPage):
    #upload and analyze multiple images
    def analyze(self, reuslts):
        #this function will format and output the results correctly. 
    def detect_face_show(self, fpath):
        # this function needs to loop through all of the files in the given folder and analyze each of their results, 
        # storing in a csv file. if there are multiple faces, it will not analyze.
    def get_image_files(self):
        
        dialog = QFileDialog()
        dialog.setOption(dialog.DontUseNativeDialog, True)
        file_name = dialog.getExistingDirectory(self, "Select A Folder")
        file = os.path.join(file_name)
        self.detect_face_show(file)
        print(file)
        
    def __init__(self, parent=None):
        super(Page3Batch, self).__init__(parent)
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()
        self.title_label = QLabel('Upload Your Images Below', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 10))
        self.title_label.adjustSize()
        
        self.button1 = QPushButton("Select Your Folder of Images to Upload Here", self)   
        self.button1.clicked.connect(self.get_image_files)
        self.button1.move(50, 150)
        
#results
class Page4Single(QtWidgets.QWizardPage):
    def __init__(self, parent=None):
        super(Page4Single, self).__init__(parent)
        
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()

        self.title_label = QLabel('Results', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()
        self.title_label = QLabel('Here are the results for the IMAGE you uploaded:', self)
        self.title_label.move(100, 125)
        self.title_label.setFont(QFont('Arial', 10))
        self.title_label.adjustSize()
        
class Page4Batch(QtWidgets.QWizardPage):
    def __init__(self, parent=None):
        super(Page4Batch, self).__init__(parent)
        
        self.title_label = QLabel('DCiFR', self)
        self.title_label.move(50, 30)
        self.title_label.setFont(QFont('Arial', 20))
        self.title_label.adjustSize()

        self.title_label = QLabel('Results', self)
        self.title_label.move(100, 75)
        self.title_label.setFont(QFont('Arial', 15))
        self.title_label.adjustSize()
        self.title_label = QLabel('Here are the results for the IMAGES you uploaded:', self)
        self.title_label.move(100, 125)
        self.title_label.setFont(QFont('Arial', 10))
        self.title_label.adjustSize()
        
if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    wizard = Wizard()
    wizard.show()
    app.exec_()   
    
#TO-DO:

# producing deepface results
# getting threshold value
# get value from checkboxes

10
20
30
40
50
60
50
47
46
44
43
41
38
37
35
34
32
31
29
28
26
25
24
22
21
19
18
16
15
13
12
10
9
